In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from bs4 import BeautifulSoup

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Define Graphing Function

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1 - Extracting Tesla Stock Data Using yfinance


Download historical data for a Tesla

In [3]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

Display the downloaded data

In [4]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2 - Extracting Tesla Revenue Data Using Webscraping

Send an HTTP request to the web page

In [5]:
url = " https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
data  = requests.get(url).text

Parsing the data using the BeautifulSoup library

In [6]:
soup = BeautifulSoup(data, 'html.parser')

Create an empty dataframe

In [7]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

Use a BeautifulSoup method for extracting data¶

In [8]:
# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for table in soup.find_all('table'):
    if table.find('th').getText().startswith("Tesla Quarterly Revenue"):
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            if len(col) != 2: continue
            Date = col[0].text
            Revenue = col[1].text
    
            # Finally we append the data of each row to the table
            tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame({"Date":[Date], "Revenue":[Revenue]})], ignore_index=True)    

Remove the comma and dollar sign from the Revenue column and remove an null or empty strings in the Revenue column.

In [9]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$',"")
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',',".")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Print the extracted data

In [10]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## Question 3 - Extracting GameStop Stock Data Using yfinance

Download historical data for a Tesla

In [11]:
gamestop = yf.Ticker("GME")
gamestop_data = gamestop.history(period="max")

Display the downloaded data

In [12]:
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666417,1.666417,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [13]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
data  = requests.get(url).text

Parsing the data using the BeautifulSoup library

In [14]:
soup = BeautifulSoup(data, 'html.parser')

Create an empty dataframe

In [15]:
gamestop_revenue = pd.DataFrame(columns=["Date", "Revenue"])

Use a BeautifulSoup method for extracting data¶

In [16]:
# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for table in soup.find_all('table'):
    if table.find('th').getText().startswith("GameStop Quarterly Revenue"):
        for row in table.find("tbody").find_all("tr"):
            col = row.find_all("td")
            if len(col) != 2: continue
            Date = col[0].text
            Revenue = col[1].text
    
            # Finally we append the data of each row to the table
            gamestop_revenue = pd.concat([gamestop_revenue,pd.DataFrame({"Date":[Date], "Revenue":[Revenue]})], ignore_index=True)    

Remove the comma and dollar sign from the Revenue column and remove an null or empty strings in the Revenue column.

In [17]:
gamestop_revenue["Revenue"] = gamestop_revenue['Revenue'].str.replace('$',"")
gamestop_revenue["Revenue"] = gamestop_revenue['Revenue'].str.replace(',',".")
gamestop_revenue.dropna(inplace=True)
gamestop_revenue = gamestop_revenue[gamestop_revenue['Revenue'] != ""]

Print the extracted data

In [18]:
gamestop_revenue.tail()

,Date,Revenue
57,2006-01-31,1.667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## Question 5 - Tesla Stock and Revenue Dashboard

Plot Tesla Stock Graph

In [19]:
make_graph(tesla_data, tesla_revenue, 'TESLA')

/var/folders/mn/sfbzswcd1s16ptkhdhpq81cr0000gp/T/ipykernel_11783/1276540637.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/var/folders/mn/sfbzswcd1s16ptkhdhpq81cr0000gp/T/ipykernel_11783/1276540637.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



## Question 6 - GameStop Stock and Revenue Dashboard

Plot Gamestop Stock Graph

In [20]:
make_graph(gamestop_data, gamestop_revenue, 'GameStop')

/var/folders/mn/sfbzswcd1s16ptkhdhpq81cr0000gp/T/ipykernel_11783/1276540637.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/var/folders/mn/sfbzswcd1s16ptkhdhpq81cr0000gp/T/ipykernel_11783/1276540637.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



## Question 7 - Sharing your Assignment Notebook

The Notebook can be found at: https://github.com/pikay7/IBM-Data-Science


## Author
Philipp Korous